# Optimizer for Constrained Functions R² -> R

Simple Python Implementation of Newton/Gradient methods.

## Objective

Given an input of a point in the R² domain, an objective function, a direction of optmization(max, min), a list of constraints k and arbitraty numbers(number small enough and iterations limit) at each iteration finding a better point until find the best one, if possible.



## Specifications


With the inputs the following steps are made:



### 1 - Check if there's a Feasible Region

In [2]:
def checkFeasibility(constraintsList):
    #todo

SyntaxError: unexpected EOF while parsing (2682153486.py, line 2)

### 2 - Iterate over 

In [ ]:
def nextPoint(currentPoint, objectiveFunction, minOrMax, constraintsList, kwargs):
    if (kwargs.iterationLimit = 0): 
        return currentPoint, "Iteration Limit Reached"
    evaluation = #todo
    if (kwargs.backEvaluation - evaluation <= kwargs.epsolon*minOrMax && kwargs.stepSize <= kwargs.epsolon): 
        return currentPoint, ""
    direction = #todo
    stepSize = #todo
    newPoint = #todo
    return nextPoint(newPoint, objectiveFunction, minOrMax, constraintsList, kwargs"""modfyied""")

## Some Functions

## Some Constraints


## Results
